In [1]:

import pandas as pd
import CU_POLARIS_Postprocessor
from CU_POLARIS_Postprocessor import clean, parallel, postprocessing
import importlib
from CU_POLARIS_Postprocessor import prerun
import CU_POLARIS_Postprocessor.clean
import CU_POLARIS_Postprocessor.config
import CU_POLARIS_Postprocessor.parallel
import CU_POLARIS_Postprocessor.postprocessing
import CU_POLARIS_Postprocessor.power_bi_processing.prep_utils
import CU_POLARIS_Postprocessor.prerun
importlib.reload(CU_POLARIS_Postprocessor.postprocessing)
importlib.reload(CU_POLARIS_Postprocessor.parallel)
importlib.reload(CU_POLARIS_Postprocessor.prerun)
importlib.reload(CU_POLARIS_Postprocessor.clean)
importlib.reload(CU_POLARIS_Postprocessor)
importlib.reload(CU_POLARIS_Postprocessor.config)
importlib.reload(CU_POLARIS_Postprocessor.power_bi_processing.prep_utils)
from CU_POLARIS_Postprocessor.config import PostProcessingConfig
from CU_POLARIS_Postprocessor.power_bi_processing import prep_utils


In [ ]:

config = PostProcessingConfig()
prerun.pre_run_checks(config)
parallel.parallel_process_folders(config)

In [32]:
from pathlib import Path
config_wtp = PostProcessingConfig(base_dir=Path("C:/Users/jpaul4/Box/Research/Papers/4_WTP Factors Paper/cases_only_critical"),output_h5=True)
prerun.pre_run_checks(config_wtp)


True

In [96]:
parallel.parallel_process_folders(config_wtp)

starting on directory C:\Users\jpaul4\Box\Research\Papers\4_WTP Factors Paper\cases_only_critical\atx_d
starting on directory C:\Users\jpaul4\Box\Research\Papers\4_WTP Factors Paper\cases_only_critical\atx_du
starting on directory C:\Users\jpaul4\Box\Research\Papers\4_WTP Factors Paper\cases_only_critical\atx_dul
starting on directory C:\Users\jpaul4\Box\Research\Papers\4_WTP Factors Paper\cases_only_critical\atx_dulp
starting on directory C:\Users\jpaul4\Box\Research\Papers\4_WTP Factors Paper\cases_only_critical\atx_dulpt
starting on directory C:\Users\jpaul4\Box\Research\Papers\4_WTP Factors Paper\cases_only_critical\atx_dulptas
starting on directory C:\Users\jpaul4\Box\Research\Papers\4_WTP Factors Paper\cases_only_critical\atx_dult
starting on directory C:\Users\jpaul4\Box\Research\Papers\4_WTP Factors Paper\cases_only_critical\atx_heur
starting on directory C:\Users\jpaul4\Box\Research\Papers\4_WTP Factors Paper\cases_only_critical\gvl_d
starting on directory C:\Users\jpaul4\Box\

c:\polaris\CU_POLARIS_Postprocessor\CU_POLARIS_Postprocessor\parallel.py:40: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->Index(['mode', 'type', 'folder'], dtype='object')]

  print(f"starting on directory {dir}")


True

## Load in Output DF for development

In [38]:
loaded_df_dict = {}
with pd.HDFStore(config_wtp.base_dir.as_posix()+'/results.h5') as store:
    for key in store.keys():
        clean_key = key.lstrip('/').strip("'")
        loaded_df_dict[clean_key] = store[key]
for key, value in loaded_df_dict.items():
    print(key)

bus_avo
distance_tnc_dist
elder_demo
fare_sensitivity_demograpic_tnc_stats
fare_sensitivity_results
fare_sensitivity_results_vo
fare_sensitivity_results_zonal
mode_Distribution_ADULT
mode_Distribution_ADULT_Counts
mode_Distribution_ADULT_Distance
pr_avo
requests_sum
tnc_results_discount
tnc_skim_demo
transit_trip_max_load


In [90]:
prep_utils.process_folder_names(config_wtp)

In [57]:
for key, value in config_wtp.results.items():
    print(key)

transit_trip_max_load
mode_Distribution_ADULT_Counts
mode_Distribution_ADULT_Distance
bus_avo
pr_avo
fare_sensitivity_results
mode_Distribution_ADULT
distance_tnc_dist
fare_sensitivity_results_zonal
fare_sensitivity_demograpic_tnc_stats
fare_sensitivity_results_vo
tnc_results_discount
elder_demo
requests_sum
tnc_skim_demo


In [98]:
mda_c_df = config_wtp.results['mode_Distribution_ADULT_Counts']
bus_avo_df = config_wtp.results['bus_avo']
pr_avo_df = config_wtp.results['pr_avo']

mode_enum = pd.read_csv('CU_POLARIS_Postprocessor\power_bi_processing\dummy_files\mode_enum.csv')

mda_c_df = mda_c_df.join(mode_enum,'mode','left')
mda_c_df.drop(columns=['mode','Enum'],inplace=True)

mode_cnt_combo = bus_avo_df.merge(pr_avo_df,left_on='folder',right_on='folder',how='left')
mode_cnt_combo = mode_cnt_combo.rename(columns={
    'cnt_pool_y': 'pr.cnt_pool', 
    'dist_pool_y': 'pr.dist_pool',
    'cnt_solo_y': 'pr.cnt_solo', 
    'dist_solo_y': 'pr.dist_solo', 
    'cnt_empt_y': 'pr.cnt_empt', 
    'dist_empt_y': 'pr.dist_empt',
    'cnt_pool_x': 'cnt_pool', 
    'dist_pool_x': 'dist_pool',
    'cnt_solo_x': 'cnt_solo', 
    'dist_solo_x': 'dist_solo', 
    'cnt_empt_x': 'cnt_empt', 
    'dist_empt_x': 'dist_empt'    
})
mode_cnt_combo['dist_mass']=mode_cnt_combo['dist_mass'].fillna(0)
mode_cnt_combo['dist_pool'].fillna(0)
mode_cnt_combo.drop(columns=['trip_avo', 'cnt_mass', 'cnt_pool', 'cnt_solo', 'cnt_empt', 
                          'pr.cnt_pool', 'pr.cnt_solo', 'pr.cnt_empt'], inplace=True)

### pivot
mode_cnt_pivot=mda_c_df.drop(columns=["HBW", "HBO", "NHB"])
mode_cnt_pivot = mode_cnt_pivot.rename(columns={"Mode": "mode"})
mode_cnt_pivot = mode_cnt_pivot.drop(columns=["age_class"])
mode_cnt_pivot = mode_cnt_pivot.pivot_table(index=mda_c_df['folder'], columns='mode', values='total', aggfunc='sum').reset_index()

mode_cnt_combo = mode_cnt_combo.merge(mode_cnt_pivot, on='folder', how='left')



mode_cnt_combo.head(10)

<>:5: SyntaxWarning: invalid escape sequence '\p'
<>:5: SyntaxWarning: invalid escape sequence '\p'
C:\Users\jpaul4\AppData\Local\Temp\ipykernel_8412\1980033208.py:5: SyntaxWarning: invalid escape sequence '\p'
  mode_enum = pd.read_csv('CU_POLARIS_Postprocessor\power_bi_processing\dummy_files\mode_enum.csv')
C:\Users\jpaul4\AppData\Local\Temp\ipykernel_8412\1980033208.py:25: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  mode_cnt_combo['dist_mass'].fillna(0, inplace=True)
C:\Users\jpaul4\AppData\Local\Temp\ipykernel_8412\19800

,mileage_avo,dist_mass,dist_pool,dist_solo,dist_empt,folder,pr.dist_pool,pr.dist_solo,pr.dist_empt,BICYCLE,BUS,RAIL,SOV,TAXI,WALK
0,4.844052,13491.603906,11729.912922,NaN,24711.834955,atx_d_7,253741.424141,426587.287120,130409.028157,142716.0,46592.0,5880.0,6188012.0,90432.0,507988.0
1,4.711998,12973.651523,11710.730734,NaN,25248.969526,atx_du_7,245923.253909,429968.703548,123763.307202,142444.0,45756.0,6060.0,6172284.0,91096.0,508336.0
2,4.814043,13372.862103,11938.919698,NaN,24621.569982,atx_dul_7,227217.593604,440913.749909,131035.845967,144248.0,46408.0,5868.0,6186712.0,90568.0,510160.0
3,4.820152,13221.560924,11815.341117,NaN,24896.449742,atx_dulp_7,232523.479972,439195.578602,131752.882443,142316.0,46776.0,6044.0,6174552.0,90864.0,508196.0
4,4.835594,13241.322797,11848.454626,NaN,24843.574360,atx_dulpt_7,221382.003689,431477.322242,135129.774637,142636.0,46784.0,5892.0,6170552.0,89336.0,509500.0
5,4.823176,13131.712617,11849.592668,NaN,24952.046498,atx_dulptas_7,247226.797847,418092.515235,123647.628608,143740.0,46800.0,5840.0,6173468.0,90440.0,510988.0
6,4.758823,13287.399494,11753.381972,NaN,24892.570317,atx_dult_7,228796.064193,430729.321011,131369.196774,142372.0,46800.0,5872.0,6179068.0,89840.0,508900.0
7,5.344188,14714.274858,12081.809768,NaN,23137.267157,atx_heur_7,16655.853005,235890.029647,33507.001283,148924.0,51508.0,5500.0,6180420.0,46748.0,507060.0
8,0.000000,0.000000,0.000000,NaN,5156.883361,gvl_d_7,2984.038037,7129.730371,1327.222639,10086.0,NaN,NaN,16596.0,2413.0,54415.0
9,0.000000,0.000000,0.000000,NaN,5156.883361,gvl_du_7,3858.679837,6434.558983,1097.435854,10194.0,NaN,NaN,16183.0,2448.0,54200.0


### Statistical Tests

In [3]:
import sqlite3
from pathlib import Path
import scipy
from CU_POLARIS_Postprocessor.utils import get_highest_iteration_folder
import pandas as pd

In [25]:
db_path = Path("C:/Users/jpaul4/Box/Research/Papers/4_WTP Factors Paper/cases_only_critical/atx_d")
iter_path = get_highest_iteration_folder(db_path)
script = f"""select 
assigned_vehicle,
tnc_request_id,
(pickup_time-request_time)/60 as wait_min, 
(dropoff_time-pickup_time)/60 as ttime, 
discount, 
fare,
case when (max_pass - party_size) > 0 then 1 else 0 end as pooled,
eVMT,
occupied_VMT,
VMT,
passengers,
trips,
mileage_AVO,
mileage_rAVO,
trip_AVO,
trip_rAVO,
pooled_service,
operating_cost,
fare-discount-operating_cost as revenue

from tnc_request a left join 
(select request,
eVMT,
occupied_VMT,
VMT,
VMT*0.50 as operating_cost,
passengers,
trips,
passengers*1.0/VMT as mileage_AVO,
passengers*1.0/occupied_VMT as mileage_rAVO,
passengers*1.0/trips as trip_AVO,
passengers*1.0 as trip_rAVO,
max_pass
from
(SELECT
    request,
    SUM(CASE WHEN trip_avo = 0 THEN travel_distance ELSE 0 END) AS eVMT,
    SUM(CASE WHEN trip_avo > 0 THEN travel_distance ELSE 0 END) AS occupied_VMT,
    SUM(travel_distance) as VMT,
    SUM(trip_avo) as passengers,
    COUNT(TNC_Trip_id_int) as trips,
    MAX(trip_avo) as max_pass
FROM (
    SELECT 
        tnc_trip_id_int, 
        request, 
        passengers*1.0/travel_distance AS mileage_avo, 
        passengers AS trip_avo, 
        travel_distance/1609.34 as travel_distance
    FROM tnc_trip
) AS subquery
GROUP BY request
ORDER BY request)) b

on a.tnc_request_id= b.request;"""

folder_name = "atx_d"

with  sqlite3.connect(iter_path.as_posix()+'/campo-Demand.sqlite') as conn:
    df = pd.read_sql(script,conn).assign(folder=folder_name)

served_requests = df.dropna(subset=['assigned_vehicle'])
columns_to_sum = [
    'wait_min',
    'ttime',
    'discount',
    'fare',
    'pooled',
    'eVMT',
    'occupied_VMT',
    'VMT',
    'mileage_AVO',
    'mileage_rAVO',
    'trip_AVO',
    'trip_rAVO',
    'operating_cost',
    'revenue'
    ]
summary_df = served_requests[columns_to_sum].agg(['mean', 'std', 'count']).transpose()

# Rename columns for clarity
summary_df.columns = ['Mean', 'StdDev', 'SampleSize']
summary_df = summary_df.assign(folder=folder_name)
summary_df.to_csv(iter_path.as_posix() + '/tnc_stat_summary.csv')

## Working

In [2]:
import importlib
import CU_POLARIS_Postprocessor
from CU_POLARIS_Postprocessor import parallel, postprocessing, prerun
from CU_POLARIS_Postprocessor.config import PostProcessingConfig
from pathlib import Path

importlib.reload(CU_POLARIS_Postprocessor.power_bi_processing.prep_utils)
from CU_POLARIS_Postprocessor.power_bi_processing.prep_utils import process_folder_names, process_tnc_ttests
import CU_POLARIS_Postprocessor.power_bi_processing
import CU_POLARIS_Postprocessor.power_bi_processing.prep_utils

config_wtp = PostProcessingConfig(
    base_dir=Path("C:/Users/jpaul4/Box/Research/Papers/4_WTP Factors Paper/cases_only_critical"),
    fresh_start=False,
    reset_sql=False,
    reset_csvs=False,reset_stops=False)

In [3]:
prerun.pre_run_checks(config_wtp)

True

In [4]:
parallel.parallel_process_folders(config_wtp)

starting on directory C:\Users\jpaul4\Box\Research\Papers\4_WTP Factors Paper\cases_only_critical\atx_d
starting on directory C:\Users\jpaul4\Box\Research\Papers\4_WTP Factors Paper\cases_only_critical\atx_du
starting on directory C:\Users\jpaul4\Box\Research\Papers\4_WTP Factors Paper\cases_only_critical\atx_dul
starting on directory C:\Users\jpaul4\Box\Research\Papers\4_WTP Factors Paper\cases_only_critical\atx_dulp
starting on directory C:\Users\jpaul4\Box\Research\Papers\4_WTP Factors Paper\cases_only_critical\atx_dulpt
starting on directory C:\Users\jpaul4\Box\Research\Papers\4_WTP Factors Paper\cases_only_critical\atx_dulptas
starting on directory C:\Users\jpaul4\Box\Research\Papers\4_WTP Factors Paper\cases_only_critical\atx_dult
starting on directory C:\Users\jpaul4\Box\Research\Papers\4_WTP Factors Paper\cases_only_critical\atx_heur
starting on directory C:\Users\jpaul4\Box\Research\Papers\4_WTP Factors Paper\cases_only_critical\gvl_d
starting on directory C:\Users\jpaul4\Box\

True

In [36]:
stat_sum = config_wtp.results['tnc_stat_summary']

## Power BI T-Test Post Processing

In [5]:
for key, value in config_wtp.results.items():
    print(key)

config_wtp.results['tnc_stat_summary']

transit_trip_max_load
mode_Distribution_ADULT_Counts
mode_Distribution_ADULT_Distance
bus_avo
pr_avo
fare_sensitivity_results
mode_Distribution_ADULT
distance_tnc_dist
fare_sensitivity_results_zonal
fare_sensitivity_demograpic_tnc_stats
fare_sensitivity_results_vo
tnc_results_discount
elder_demo
requests_sum
tnc_stat_summary
tnc_skim_demo


,Metric,Mean,StdDev,SampleSize,folder
0,wait_min,12.745160,25.743033,22612.0,atx_d_7
1,ttime,24.660025,48.693621,22612.0,atx_d_7
2,discount,44.053894,84.495512,22612.0,atx_d_7
3,fare,75.472986,148.686354,22612.0,atx_d_7
4,pooled,0.413586,0.492487,22612.0,atx_d_7
...,...,...,...,...,...
219,mileage_rAVO,0.494349,1.216495,2322.0,gvl_heur_7
220,trip_AVO,0.598501,0.318689,2324.0,gvl_heur_7
221,trip_rAVO,1.155766,0.523907,2324.0,gvl_heur_7
222,operating_cost,2.426092,0.841576,2324.0,gvl_heur_7


In [5]:
base_cases = ['atx_du_7','gvl_du_7']
process_tnc_ttests(config_wtp,base_cases)

In [6]:
for key, value in config_wtp.results.items():
    print(key)

transit_trip_max_load
mode_Distribution_ADULT_Counts
mode_Distribution_ADULT_Distance
bus_avo
pr_avo
fare_sensitivity_results
mode_Distribution_ADULT
distance_tnc_dist
fare_sensitivity_results_zonal
fare_sensitivity_demograpic_tnc_stats
fare_sensitivity_results_vo
tnc_results_discount
elder_demo
requests_sum
tnc_stat_summary
tnc_skim_demo
tnc_ttests


In [7]:
config_wtp.results['tnc_ttests']

,city,base_case,comparison_case,metric,t-statistic,p-value
0,atx,atx_du_7,atx_d_7,wait_min,-4.665133,3.092959e-06
1,atx,atx_du_7,atx_d_7,ttime,-0.331545,7.402341e-01
2,atx,atx_du_7,atx_d_7,discount,1.308973,1.905502e-01
3,atx,atx_du_7,atx_d_7,fare,-1.007463,3.137176e-01
4,atx,atx_du_7,atx_d_7,pooled,-1.516308,1.294483e-01
...,...,...,...,...,...,...
219,gvl,gvl_du_7,gvl_heur_7,mileage_rAVO,11.361113,1.562706e-29
220,gvl,gvl_du_7,gvl_heur_7,trip_AVO,34.309489,9.123050e-231
221,gvl,gvl_du_7,gvl_heur_7,trip_rAVO,30.658850,1.421762e-188
222,gvl,gvl_du_7,gvl_heur_7,operating_cost,-3.421071,6.289901e-04
